In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT = 480

ANSWERED_CALLS = 1000
FINISHED_T = 0
all_customers = []
operation_t = [0,0]
shift_number = 0
is_done = 0
unsatisfied_customers = 0

In [3]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range)
        operation_t[1] += time
        yield env.timeout(time)

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.waiting_t = 0
        
    
    def call(self):
        global is_done
        global unsatisfied_customers
#         print('%s initiated a call at %g' % (self.name, self.env.now))
        
        
        # Voice recognition and routing
        if router.is_idle():
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        yield self.env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router.count.remove(self.name)
        if router.is_idle():
            router.idle_t += [self.env.now]
        
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
#             print('%s hangs up the call at %g' % (self.name, self.env.now))
            #all_customers.append(self)
            self.operator = None
            is_done += 1
            unsatisfied_customers += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()
            return 
        
#         print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
            self.waiting_t = self.env.now
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
#                 print('%s is left the operator %d after 10 minutes at %g' % (self.name,
#                                                                (self.operator==operator2)+1,
#                                                                self.env.now))
                #all_customers.append(self)
                self.operator = None
                is_done += 1
                unsatisfied_customers += 1
                if is_done == ANSWERED_CALLS:
                    finished.succeed()
                return 
            self.waiting_t = self.env.now - self.waiting_t
#             print('%s is assigned to the operator %d at %g' % (self.name,
#                                                                (self.operator==operator2)+1,
#                                                                self.env.now))
            yield self.env.process(service(env,self.operator))
#             print('%s is done with operator %d at %g' % (self.name,
#                                                         (self.operator==operator2)+1,
#                                                         self.env.now))
            #all_customers.append(self)
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()

In [5]:
class robocall:
    def __init__(self,env):
        self.count = []
        self.working_t = []
        self.idle_t = [0]
        self.env = env
        
    def answer(self):
        return len(self.count) < 100
    
    def is_idle(self):
        return len(self.count) == 0
            
    def utilization(self):
        return sum([ w-i for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

In [6]:
def breaks():
    return [random.uniform(0,SHIFT) for i in  range(np.random.poisson(8))]

In [7]:
def shift(env):
    global shift_number
    shift_number += 1
    
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)
#     print("the time %g" % env.now)

    env.process(shift(env))

In [8]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
#                 print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
#                 print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
#             print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [9]:
def customer_generator(env):
    start = True
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            all_customers.append(customer)
            
    FINISHED_T = env.now 

In [10]:
env = simpy.Environment()

finished = Event(env)
router = robocall(env)

operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

# env.process(customer_generator(env))
# env.run(finished) 
# FINISHED_T = env.now
# print(FINISHED_T)
# print(len(all_customers))

In [11]:
def simulation(CALLS=1000, seed=978, verbose=0):
    
    global FINISHED_T 
    global all_customers
    global operation_t
    global shift_number
    global ANSWERED_CALLS
    global is_done
    global unsatisfied_customers
    
    unsatisfied_customers = 0
    is_done = 0
    ANSWERED_CALLS = CALLS
    FINISHED_T = 0
    all_customers = []
    operation_t = [0,0]
    shift_number = 0
    random.seed(seed)
    
    global  env 
    global finished 
    global router 
    global operator1 
    global operator2 
    env = simpy.Environment()
    finished = Event(env)
    router = robocall(env)
    operator1 = simpy.PriorityResource(env, capacity = 1)
    operator2 = simpy.PriorityResource(env, capacity = 1)
    env.process(customer_generator(env))
    env.run(finished) 
    FINISHED_T = env.now
    
    
    customer_waiting = [c.waiting_t for c in all_customers if c.operator != None]
    if verbose:
        print('Voice Recognition System Utilization: ', router.utilization())
        print('Utilization Operator 1:', operation_t[0]/FINISHED_T, 
              '\nUtilization Operator 2:' ,operation_t[1]/FINISHED_T)
        print('Average Waiting Time: ', sum(customer_waiting)/len(customer_waiting))
        
    return (router.utilization(),  
            operation_t[0]/FINISHED_T, 
            operation_t[1]/FINISHED_T, 
            sum(customer_waiting)/len(customer_waiting),
            unsatisfied_customers)

In [12]:
%%time
seeds = [305, 451, 973, 867, 135, 231, 53, 199, 155, 747]
calls = [1000, 5000]
statistics_1000 = [simulation(1000,s) for s in seeds]
statistics_5000 = [simulation(5000,s) for s in seeds]

Wall time: 4.75 s


In [13]:
print("Statistics for 1000 caller:\n")
for i in statistics_1000:
    for j in range(len(i)):
        if j == 0:
            print("Utilization of the answering system:", i[j])
        elif j == 1:
            print('Utilization Operator 1:', i[j], 
              '\nUtilization Operator 2:' , i[j+1])
        elif j == 3:
            print("Average Total Waiting Time:", i[j])
        elif j == 4:
            print("Number of unsatisfied customers:", i[j])
    print()
print("Maximum Total Waiting Time to Total System Time Ratio:", max([i[3] for i in statistics_1000]), "\n")
print("Average number of people waiting to be served by Operator 1:?????",
      "\nAverage number of people waiting to be served by Operator 2:?????\n")
print("Average unsatisfied customers for 10 simulation:", sum([i[4] for i in statistics_1000])/10)

Statistics for 1000 caller:

Utilization of the answering system: 0.4323498441645064
Utilization Operator 1: 0.4242313236100745 
Utilization Operator 2: 0.42881152957903745
Average Total Waiting Time: 1.5732265010652766
Number of unsatisfied customers: 148

Utilization of the answering system: 0.399613043810191
Utilization Operator 1: 0.4429927111127455 
Utilization Operator 2: 0.41998859959114343
Average Total Waiting Time: 1.6160897355494974
Number of unsatisfied customers: 141

Utilization of the answering system: 0.4623814570180583
Utilization Operator 1: 0.4331222581571472 
Utilization Operator 2: 0.3863257808805934
Average Total Waiting Time: 1.6807817740296698
Number of unsatisfied customers: 157

Utilization of the answering system: 0.45129221401189923
Utilization Operator 1: 0.4476875869565421 
Utilization Operator 2: 0.40991312986183714
Average Total Waiting Time: 1.6373526210432938
Number of unsatisfied customers: 143

Utilization of the answering system: 0.44401570626994696

In [14]:
print("Statistics for 5000 caller:\n")
for i in statistics_5000:
    for j in range(len(i)):
        if j == 0:
            print("Utilization of the answering system:", i[j])
        elif j == 1:
            print('Utilization Operator 1:', i[j], 
              '\nUtilization Operator 2:' , i[j+1])
        elif j == 3:
            print("Average Total Waiting Time:", i[j])
        elif j == 4:
            print("Number of unsatisfied customers:", i[j])
    print()
print("Maximum Total Waiting Time to Total System Time Ratio:", max([i[3] for i in statistics_1000]), "\n")
print("Average number of people waiting to be served by Operator 1:?????",
      "\nAverage number of people waiting to be served by Operator 2:?????\n")
print("Average unsatisfied customers for 10 simulation:", sum([i[4] for i in statistics_1000])/10)

Statistics for 5000 caller:

Utilization of the answering system: 0.43256770198231526
Utilization Operator 1: 0.43393551170608674 
Utilization Operator 2: 0.41795292483774843
Average Total Waiting Time: 1.69108348204805
Number of unsatisfied customers: 736

Utilization of the answering system: 0.42068780442967535
Utilization Operator 1: 0.40924684845408293 
Utilization Operator 2: 0.42025527561925813
Average Total Waiting Time: 1.6776514573112733
Number of unsatisfied customers: 736

Utilization of the answering system: 0.4371883825861248
Utilization Operator 1: 0.42543901367061715 
Utilization Operator 2: 0.41913875519593874
Average Total Waiting Time: 1.6588854826211328
Number of unsatisfied customers: 767

Utilization of the answering system: 0.4268119305425684
Utilization Operator 1: 0.44531138676207443 
Utilization Operator 2: 0.41002977877667546
Average Total Waiting Time: 1.6608457244707087
Number of unsatisfied customers: 738

Utilization of the answering system: 0.432208867273